In [3]:
import pandas as pd
import numpy as np
import requests

from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [2]:
url = "https://yeniemlak.az/bakida-evlerin-qiymeti?t=2018"
html = requests.get(url).content
tables = pd.read_html(html)

dateMap = {
    "Yanvar": "1",
    "Fevral": "2",
    "Mart": "3",
    "Aprel": "4",
    "May": "5",
    "İyun": "6",
    "İyul": "7",
    "Avgust": "8",
    "Sentyabr": "9",
    "Oktyabr": "10",
    "Noyabr": "11",
    "Dekabr": "12"
}


print("len of tables: ", len(tables))

len of tables:  26


In [3]:
tableMap = {}
year = 2023
q = 2

for index, table in enumerate(tables):
  if(index in (0, 1, 2)):
    continue
  if(year <= 2015):
    table = table.iloc[4:]
  else:
    table = table.iloc[5:]
  n_rows = len(table)
  pattern = np.tile([True, True, True, False], n_rows // 4 + 1)[:n_rows]
  tableMap[str(f"{year}_{q}")] = table[pattern]
  if(q == 2):
    tableMap[str(f"{year}_{q}")] = tableMap[str(f"{year}_{q}")].rename(columns={0: 'City', 1: 'Type', 2: f'July_{year}', 3: f'August_{year}', 4: f'September_{year}', 5: f'October_{year}', 6: f'November_{year}', 7: f'December_{year}'})
  else:
    tableMap[str(f"{year}_{q}")] = tableMap[str(f"{year}_{q}")].rename(columns={0: 'City', 1: 'Type', 2: f'January_{year}', 3: f'February_{year}', 4: f'March_{year}', 5: f'April_{year}', 6: f'May_{year}', 7: f'June_{year}'})

  year = year - 1 if q == 1 else year
  q = 2 if q == 1 else 1

In [4]:
tableMap.keys()

dict_keys(['2023_2', '2023_1', '2022_2', '2022_1', '2021_2', '2021_1', '2020_2', '2020_1', '2019_2', '2019_1', '2018_2', '2018_1', '2017_2', '2017_1', '2016_2', '2016_1', '2015_2', '2015_1', '2014_2', '2014_1', '2013_2', '2013_1', '2012_2'])

In [5]:
processedMap = {}
tableMap.pop("2012_2")
not_used_cities = ["Gəncə", "Abşeron rayonu, Xırdalan, Masazır", "Sumqayıt", ]

for keys in tableMap.keys():
    if(keys[-1] == '2'):
        continue
    processedMap[keys[:-2]] = pd.concat([tableMap[keys], tableMap[f"{keys[:-1]}2"]], axis=1)
    processedMap[keys[:-2]] = processedMap[keys[:-2]].loc[:,~processedMap[keys[:-2]].columns.duplicated()].copy()
    processedMap[keys[:-2]] = processedMap[keys[:-2]][~processedMap[keys[:-2]].applymap(lambda x: x in not_used_cities).any(axis=1)]


In [6]:
keys = list(processedMap.keys())
finalDf = processedMap["2013"]

for key in keys[::-1]:
    if key == '2013':
        continue

    finalDf = pd.merge(finalDf, processedMap[key], on=['City', 'Type'], how='inner')

finalDf.fillna(method='ffill', axis=1, inplace=True)

In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

finalDf["Type"] = label_encoder.fit_transform(finalDf['Type'])

count_df = finalDf.applymap(lambda x: x.count('-') if isinstance(x, str) else 0)
total_count = count_df.sum().sum()


while total_count != 0:
  for i, row in finalDf.iterrows():
      for j in range(1, len(finalDf.columns)):
          if row[j] == '-':
            finalDf.iat[i, j] = row[j-1]
  count_df = finalDf.applymap(lambda x: x.count('-') if isinstance(x, str) else 0)
  total_count = count_df.sum().sum()


for column in finalDf.columns:
  if(column in ("City", "Type")): continue
  finalDf[column] = finalDf[column].str.replace(',', '.')
  finalDf[column] = finalDf[column].astype(str).str.replace('\xa0', ' ', regex=False)
  finalDf[column] = finalDf[column].str.replace(' ', '')
  finalDf[column] = finalDf[column].astype(float)


count_df = finalDf.applymap(lambda x: x.count('-') if isinstance(x, str) else 0)

total_count = count_df.sum().sum()

print("Total '-' count across the DataFrame:", total_count)

Total '-' count across the DataFrame: 0


In [13]:
from sklearn.preprocessing import MinMaxScaler


def filter_data(city, type_, df):
  return df[(df['City'] == city) & (df['Type'] == type_)].select_dtypes(include=[np.number])

city = 'Nərimanov'
type_ = 0
filtered_df = filter_data(city, type_, finalDf)

data = filtered_df.values
data = data.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

n_input = 12
n_features = 1

generator = TimeseriesGenerator(data_scaled, data_scaled, length=n_input, batch_size=1)

model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_input, n_features)),
    Conv1D(filters=32, kernel_size=2, activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1)
])


model.compile(optimizer='nadam', loss='mse')


model.fit(generator, epochs=200, verbose=1)

Epoch 1/200
121/121 [==============================] - 2s 3ms/step - loss: 0.0192
Epoch 2/200
121/121 [==============================] - 0s 3ms/step - loss: 0.0015
Epoch 3/200
121/121 [==============================] - 0s 3ms/step - loss: 0.0013
Epoch 4/200
121/121 [==============================] - 0s 3ms/step - loss: 0.0012
Epoch 5/200
121/121 [==============================] - 0s 3ms/step - loss: 0.0010
Epoch 6/200
121/121 [==============================] - 0s 3ms/step - loss: 8.8998e-04
Epoch 7/200
121/121 [==============================] - 0s 3ms/step - loss: 9.0481e-04
Epoch 8/200
121/121 [==============================] - 0s 3ms/step - loss: 6.6594e-04
Epoch 9/200
121/121 [==============================] - 0s 2ms/step - loss: 5.6764e-04
Epoch 10/200
121/121 [==============================] - 0s 3ms/step - loss: 6.5537e-04
Epoch 11/200
121/121 [==============================] - 0s 3ms/step - loss: 5.7114e-04
Epoch 12/200
121/121 [==============================] - 0s 3ms/step - lo

In [16]:
model.save('model.h5')

In [19]:
filtered_df = filter_data(city, type_, finalDf)

city = 'Yasamal'
type_ = 0
data = filtered_df.values
data = data.reshape(-1, 1)
print(data)
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)
print(data_scaled)


last_sequence = data_scaled[-n_input:].reshape((1, n_input, n_features))
predicted_scaled = model.predict(last_sequence)

predicted = scaler.inverse_transform(predicted_scaled)

print(f"Predicted next value: {predicted[0][0]}")

[[   0.  ]
 [1133.55]
 [1129.33]
 [1150.94]
 [1193.44]
 [1208.61]
 [1186.02]
 [1206.26]
 [1217.31]
 [1264.55]
 [1310.07]
 [1295.14]
 [1298.33]
 [1327.83]
 [1398.14]
 [1477.51]
 [1498.31]
 [1545.24]
 [1523.28]
 [1524.02]
 [1577.93]
 [1586.5 ]
 [1584.06]
 [1578.82]
 [1543.42]
 [1495.76]
 [1495.76]
 [1545.54]
 [1547.2 ]
 [1539.31]
 [1537.89]
 [1501.46]
 [1494.15]
 [1444.38]
 [1408.28]
 [1385.13]
 [1385.13]
 [1406.23]
 [1430.6 ]
 [1445.67]
 [1448.71]
 [1439.17]
 [1437.86]
 [1434.06]
 [1436.02]
 [1392.73]
 [1399.49]
 [1440.71]
 [1451.08]
 [1454.26]
 [1472.35]
 [1479.67]
 [1468.1 ]
 [1458.64]
 [1433.35]
 [1415.63]
 [1407.83]
 [1411.97]
 [1424.94]
 [1400.01]
 [1411.73]
 [1398.01]
 [1393.85]
 [1400.71]
 [1418.02]
 [1413.02]
 [1396.82]
 [1425.52]
 [1420.61]
 [1422.69]
 [1426.46]
 [1426.35]
 [1434.  ]
 [1444.58]
 [1442.01]
 [1457.99]
 [1417.15]
 [1410.74]
 [1424.94]
 [1448.91]
 [1447.23]
 [1467.46]
 [1483.38]
 [1496.24]
 [1491.3 ]
 [1503.09]
 [1514.77]
 [1514.77]
 [1534.96]
 [1514.36]
 [1521.61]